[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2023 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

## Übung / Exercise 7 Ideal Dirac Comb Sampling and Ideal Lowpass Reconstruction for Frequency Domain Signals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import Generator, PCG64

render_pdf = True

if render_pdf == True:
    plt.rcParams.update({'font.size': 10})
    figsize = (6, 4.5)
    ms = 10
else:
    plt.rcParams.update({'font.size': 18})
    figsize = (14, 7)
    ms = 15
    
def my_sinc(x):  # we rather use definition sinc(x) = sin(x)/x, thus:
    return np.sinc(x/np.pi)

In [ ]:
plt.figure(figsize=figsize)
w = np.linspace(-12*np.pi, +12*np.pi, 2**10)
plt.plot(w / np.pi, my_sinc(w), lw=3)
plt.xlim(-12,12)
plt.xticks(np.arange(-12,12+1,1))
plt.xlabel(r'$\omega / \pi$')
plt.ylabel(r'sinc($\omega$)')
plt.grid(True)

In [ ]:
Ts = 2/3
ws = 2*np.pi/Ts
A = 4.5
print('A =', A, ', Ts =', Ts, ', ws/pi =', ws/np.pi)

In [ ]:
M = 15
nu = np.arange(-M, M+1)
w = np.arange(-M*ws, (M+1)*ws, ws / 2**6)

In [ ]:
rng = Generator(PCG64(12345678))
rgb = rng.uniform(0.1, 0.9, 3*len(nu)).reshape(len(nu), 3)

In [ ]:
def sinc_sampling_sinc_interpolation(filename):
    print('Ts=%4.3f s, ws=%4.3f rad/s, Th=%4.3f s, Th/2=%4.3f s, A Th=%4.3f / Hz' %
          (Ts, ws, Th, Th/2, A*Th))
    print('ws*Th/2=%5.4f' % (ws*Th/2))
    plt.figure(figsize=figsize)

    # fourier transform
    XFT = A*Th * my_sinc((w) * Th/2)

    # fourier series
    XFS = A*Th * my_sinc((ws*nu) * Th/2)

    # fourier transform as sinc-interpolation from fourier series
    Xr = np.zeros_like(w)
    for nui in nu:
        XFSnui = A*Th * my_sinc((ws*nui) * Th/2)
        Xrnui = XFSnui * my_sinc(np.pi*(w-nui*ws)/ws)
        plt.plot(w/np.pi, Xrnui, color=rgb[nui+M, :], lw=1.5)
        Xr += Xrnui
    # plot last Xrnui to get label
    plt.plot(w/np.pi, Xrnui, 'C7', lw=1,
             label=r'$X_\mathrm{r}(\mathrm{j}\omega)[\nu]$')
    # plot fourier transform
    plt.plot(w/np.pi, XFT, 'k', lw=3, label=r'$X(\mathrm{j}\omega)$')
    # plot reconstructed fourier transform, note that we only use finite sum
    plt.plot(w/np.pi, Xr, 'C3--', lw=2,
             label=r'$X_\mathrm{r}(\mathrm{j}\omega)$')
    # plot fourier series
    plt.plot(nu*ws/np.pi, XFS, 'C7o', ms=5)
    # we don't see these dots as they are overlayed by:
    for nui in nu:
        plt.plot(nui*ws/np.pi, XFS[nui+M], 'o', color=rgb[nui+M, :], ms=ms)
    plt.stem(nu*ws/np.pi, XFS,
             linefmt='C7:',
             markerfmt='C7o',
             basefmt='C7:', label=r'$X[\nu \omega_s]$')

    plt.xticks(np.arange(-18, 18+3, 3))
    plt.xlim(-18, +18)
    plt.xlabel(r'normalized frequency $\omega / \pi$')
    plt.title('Ts=%4.3f s, ws=%4.3f rad/s, Th=%4.3f s, Th/2=%4.3f s, A Th=%4.3f' %
              (Ts, ws, Th, Th/2, A*Th))
    plt.legend()
    plt.grid(True)
    if render_pdf is True:
        if filename is not None:
            plt.savefig(filename)
    
    plt.figure(figsize=figsize)
    t = np.linspace(-2/3,2/3,2**10)
    xr = np.zeros(2**10, dtype='complex')
    for nui in nu:
        XFSnui = A*Th * my_sinc((ws*nui) * Th/2)
        xr += 1/Ts * XFSnui * np.exp(+1j*ws*nui*t)
    plt.plot(t, np.real_if_close(xr))
    plt.plot((Th/2, Th/2),(-1, 5), 'navy', lw=4)
    plt.plot((+Ts/2, +Ts/2),(-1, 5), 'C3', lw=3)
    plt.plot((-Ts/2, -Ts/2),(-1, 5), 'C3', lw=3)
    plt.xlabel('t / s')
    plt.ylabel(r'$\tilde{x}(t)$')
    plt.title(r'signal synthesis from Fourier series coefficients $\mu$=%d...%d' % (nu[0], nu[-1]))
    plt.grid(True)
    

In [ ]:
Th = Ts/3  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation(None)

In [ ]:
# these are the chosen parameters for task 45C76AFB33
Th = Ts/2  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation('SpectrumSampling_Th_Ts1_2_45C76AFB33.pdf')

In [ ]:
# suitable for perfect reconstruction
Th = Ts*3/4  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation('SpectrumSampling_Th_Ts3_4_45C76AFB33.pdf')

In [ ]:
Th = Ts*0.9  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation(None)

In [ ]:
# critical sampling and reconstruction
# this leads to Dirac Impulse at w=0 with weight 3
# i.e. a pure DC signal of 4.5, as expected
Th = Ts  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation('SpectrumSampling_Th_Ts1_1_45C76AFB33.pdf')

In [ ]:
# undersampling case
# the reconstruction fails
Th = 4/3*Ts  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation('SpectrumSampling_Th_Ts4_3_45C76AFB33.pdf')

In [ ]:
# undersampling case
# the reconstruction fails
Th = 5/2*Ts  # Th<Ts, Th/2<Ts/2 for perfect reconstruction
sinc_sampling_sinc_interpolation(None)

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.